## Basketball Free-throw form correction using YOLOV8 and LSTM

### Approach:


1.   Use YOLOV8 Pose model to extract the XY coordinates of different joints
2.   Calculate angles the arm, legs and hip subtend with the ground
3.   Train LSTM model with angles different bones subtend with the ground per each frame
4.   Use a new video to detect faults in the shooting form



#### Install ultralytics (YOLO V8)

In [ ]:
! pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.2/663.2 kB 3.2 MB/s eta 0:00:00


In [ ]:
import cv2
import torch
import time
import imageio
from ultralytics import YOLO
import numpy as np
import pandas as pd
#add the DetectKeypoint .py file in cwd
from detection_keypoint import DetectKeypoint

### Mount the google drive

In [ ]:

from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


#### Create a list of all the videos to be extracted

In [ ]:
make_temp=['ankur-make-1','ankur-make-2','ankur-make-3','ankur-make-4','ankur-make-5','ankur-make-6','ankur-make-7','ankur-make-8','mahesh_make1','mahesh_make2','mahesh_make3','mahesh_make6','mahesh_make8','mahesh_make9','mahesh_make10','mahesh_make12','mahesh_make13','mahesh_make14','mahesh_make15','mahesh_make16','mahesh_make17','mahesh_make18','mahesh_make19','mahesh_make20','mahesh_make21','mahesh_make22','mahesh_make23','mahesh_make24','mahesh_make25']
video_list_make=[]
for i in make_temp:
  video_list_make.append(i+'.mp4')
video_list_make


['ankur-make-1.mp4',
 'ankur-make-2.mp4',
 'ankur-make-3.mp4',
 'ankur-make-4.mp4',
 'ankur-make-5.mp4',
 'ankur-make-6.mp4',
 'ankur-make-7.mp4',
 'ankur-make-8.mp4',
 'mahesh_make1.mp4',
 'mahesh_make2.mp4',
 'mahesh_make3.mp4',
 'mahesh_make6.mp4',
 'mahesh_make8.mp4',
 'mahesh_make9.mp4',
 'mahesh_make10.mp4',
 'mahesh_make12.mp4',
 'mahesh_make13.mp4',
 'mahesh_make14.mp4',
 'mahesh_make15.mp4',
 'mahesh_make16.mp4',
 'mahesh_make17.mp4',
 'mahesh_make18.mp4',
 'mahesh_make19.mp4',
 'mahesh_make20.mp4',
 'mahesh_make21.mp4',
 'mahesh_make22.mp4',
 'mahesh_make23.mp4',
 'mahesh_make24.mp4',
 'mahesh_make25.mp4']

In [ ]:
video_list_miss=[str(i)+'.mp4' for i in range(1,24)]
video_list_miss

['1.mp4',
 '2.mp4',
 '3.mp4',
 '4.mp4',
 '5.mp4',
 '6.mp4',
 '7.mp4',
 '8.mp4',
 '9.mp4',
 '10.mp4',
 '11.mp4',
 '12.mp4',
 '13.mp4',
 '14.mp4',
 '15.mp4',
 '16.mp4',
 '17.mp4',
 '18.mp4',
 '19.mp4',
 '20.mp4',
 '21.mp4',
 '22.mp4',
 '23.mp4']

#### Use YOLOv8 model to extract joint coordinates for Makes

In [ ]:
shot_array=[]
detection_keypoint = DetectKeypoint()
df_xy_coord_fin=pd.DataFrame()
joints_extracted_from_pose=34
#Download and add the yolov8n-pose.pt model to your CWD
model = YOLO('yolov8n-pose.pt')
for i in video_list_make:

  # Load the YOLO model
  xy_coord_array=np.array([])
  xy_coord_array=np.zeros(joints_extracted_from_pose)

  # Open the video file
  video_path = "/content/drive/MyDrive/Shots/Shot Makes/"+i
  print(video_path)
  cap = cv2.VideoCapture(video_path)

  # Create a video writer to save the output
  writer = imageio.get_writer("output.mp4", mode="I")

  # Loop through the video frames. Pass each frame to detection_keypoint, which extracts XY coordinates of 34 joints
  while cap.isOpened():
      # Read a frame from the video
      success, frame = cap.read()

      if success:
        start_time = time.time()
        # Run pose detection on the frame

        results = detection_keypoint(frame)
        results_keypoint = detection_keypoint.get_xy_keypoint(results)

        body_part_xy=np.array(results_keypoint)

        # Create an array with XY coordinates for each frame
        xy_coord_array=np.vstack((xy_coord_array,body_part_xy))

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF ==ord('q'):
            break
      else:
          # Break the loop if the end of the video is reached
          break

  df_xs_coord=pd.DataFrame(xy_coord_array)
  #label the video
  df_xs_coord['video']=i
  df_xs_coord['label']=1
  #number each frame
  df_xs_coord['frame_number']=range(1,len(xy_coord_array)+1)

  df_xy_coord_fin = pd.concat([df_xy_coord_fin, df_xs_coord], ignore_index=True)
  # Release the video capture object and close the display window
  cap.release()
  cv2.destroyAllWindows()

  # Close the video writer
  writer.close()


100%|██████████| 50.8M/50.8M [00:00<00:00, 163MB/s]


100%|██████████| 6.51M/6.51M [00:00<00:00, 63.5MB/s]


/content/drive/MyDrive/Shots/Shot Makes/ankur-make-1.mp4

WARNING ⚠️ NMS time limit 0.550s exceeded
0: 512x640 1 person, 135.3ms
Speed: 16.9ms preprocess, 135.3ms inference, 698.7ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 1 person, 29.7ms
Speed: 5.5ms preprocess, 29.7ms inference, 2.1ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 1 person, 29.6ms
Speed: 5.6ms preprocess, 29.6ms inference, 2.2ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 1 person, 29.6ms
Speed: 5.7ms preprocess, 29.6ms inference, 1.7ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 1 person, 29.6ms
Speed: 3.8ms preprocess, 29.6ms inference, 2.4ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 1 person, 29.6ms
Speed: 7.5ms preprocess, 29.6ms inference, 2.3ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 1 person, 25.8ms
Speed: 5.9ms preprocess, 25.8ms inference, 2.5ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 1 

#### Check for nulls

In [ ]:
df_xy_coord_fin.isna().sum()


0               0
1               0
2               0
3               0
4               0
5               0
6               0
7               0
8               0
9               0
10              0
11              0
12              0
13              0
14              0
15              0
16              0
17              0
18              0
19              0
20              0
21              0
22              0
23              0
24              0
25              0
26              0
27              0
28              0
29              0
30              0
31              0
32              0
33              0
video           0
label           0
frame_number    0
dtype: int64

#### Use YOLOv8 model to extract joint coordinates for Misses

In [ ]:
shot_array=[]
detection_keypoint = DetectKeypoint()
df_xy_coord_fin_misses=pd.DataFrame()

for i in video_list_miss:

  # Load the YOLO model
  model = YOLO('yolov8n-pose.pt')

  # Open the video file
  video_path = "/content/drive/MyDrive/Shots/Shot Misses/"+i
  print(video_path)
  cap = cv2.VideoCapture(video_path)

  # Create a video writer to save the output
  writer = imageio.get_writer("output.mp4", mode="I")
  joints_extracted_from_pose=34
  xy_coord_array=np.zeros(joints_extracted_from_pose)
  # Loop through the video frames
  while cap.isOpened():
      # Read a frame from the video
      success, frame = cap.read()

      if success:
        start_time = time.time()
        # Run pose detection on the frame

        results = detection_keypoint(frame)
        results_keypoint = detection_keypoint.get_xy_keypoint(results)

        body_part_xy=np.array(results_keypoint)


        xy_coord_array=np.vstack((xy_coord_array,body_part_xy))



        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF ==ord('q'):
            break
      else:
          # Break the loop if the end of the video is reached
          break

  # Release the video capture object and close the display window
  df_xs_coord=pd.DataFrame(xy_coord_array)
  df_xs_coord['video']=i
  df_xs_coord['label']=0
  df_xs_coord['frame_number']=range(1,len(xy_coord_array)+1)

  df_xy_coord_fin_misses = pd.concat([df_xy_coord_fin_misses, df_xs_coord], ignore_index=True)
  cap.release()
  cv2.destroyAllWindows()

  # Close the video writer
  writer.close()


/content/drive/MyDrive/Shots/Shot Misses/1.mp4

0: 480x640 1 person, 30.7ms
Speed: 4.0ms preprocess, 30.7ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 29.5ms
Speed: 3.8ms preprocess, 29.5ms inference, 9.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 29.4ms
Speed: 6.8ms preprocess, 29.4ms inference, 6.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 34.6ms
Speed: 5.7ms preprocess, 34.6ms inference, 2.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 40.0ms
Speed: 7.4ms preprocess, 40.0ms inference, 9.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 27.0ms
Speed: 3.3ms preprocess, 27.0ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 27.1ms
Speed: 4.1ms preprocess, 27.1ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 27.1ms
Speed: 7.8ms preprocess, 27.1ms inference,

In [ ]:
df_xy_coord_fin_misses

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,video,label,frame_number
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.mp4,0,1
1,0.244748,0.469072,0.242726,0.463604,0.237798,0.466227,0.0,0.0,0.224721,0.481162,...,0.698153,0.228349,0.704048,0.229006,0.773553,0.214974,0.781680,1.mp4,0,2
2,0.241856,0.469719,0.240809,0.463748,0.234975,0.466184,0.0,0.0,0.222226,0.479791,...,0.698162,0.228678,0.705289,0.228929,0.773118,0.213182,0.782527,1.mp4,0,3
3,0.240319,0.473987,0.239087,0.467859,0.233368,0.470006,0.0,0.0,0.219059,0.482814,...,0.699723,0.227879,0.709614,0.226327,0.772932,0.211209,0.787645,1.mp4,0,4
4,0.238371,0.475059,0.000000,0.000000,0.231190,0.471483,0.0,0.0,0.216147,0.485944,...,0.701566,0.229911,0.712696,0.224901,0.772323,0.208862,0.788640,1.mp4,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,0.375570,0.399020,0.372361,0.393344,0.369477,0.393789,0.0,0.0,0.351877,0.405190,...,0.641477,0.358902,0.647209,0.351591,0.734951,0.334408,0.744310,23.mp4,0,57
1096,0.375842,0.398520,0.372638,0.392600,0.369709,0.393252,0.0,0.0,0.351811,0.404797,...,0.639471,0.360213,0.645472,0.350392,0.734034,0.335498,0.741388,23.mp4,0,58
1097,0.376478,0.398737,0.372547,0.392018,0.370101,0.392954,0.0,0.0,0.351889,0.404138,...,0.638220,0.357322,0.644855,0.353428,0.732414,0.332303,0.740661,23.mp4,0,59
1098,0.376569,0.398836,0.000000,0.000000,0.369938,0.392954,0.0,0.0,0.351718,0.403736,...,0.638895,0.352830,0.644631,0.350808,0.733159,0.334628,0.740569,23.mp4,0,60


In [ ]:
df_all_coord = pd.concat([df_xy_coord_fin, df_xy_coord_fin_misses], ignore_index=True)
df_all_coord

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,video,label,frame_number
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,ankur-make-1.mp4,1,1
1,0.141942,0.519379,0.140354,0.512539,0.133615,0.515330,0.0,0.0,0.117914,0.528823,...,0.749542,0.120475,0.763546,0.123502,0.825739,0.101721,0.848228,ankur-make-1.mp4,1,2
2,0.143001,0.522868,0.140956,0.515682,0.134432,0.518720,0.0,0.0,0.117831,0.532366,...,0.750962,0.130295,0.763179,0.125134,0.828403,0.102264,0.846033,ankur-make-1.mp4,1,3
3,0.142824,0.530111,0.000000,0.000000,0.134676,0.526676,0.0,0.0,0.118476,0.540698,...,0.754452,0.135328,0.768395,0.123493,0.830647,0.105687,0.852329,ankur-make-1.mp4,1,4
4,0.145013,0.539606,0.000000,0.000000,0.137419,0.535154,0.0,0.0,0.120887,0.547287,...,0.762864,0.137136,0.777557,0.118015,0.833874,0.108978,0.855727,ankur-make-1.mp4,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2543,0.375570,0.399020,0.372361,0.393344,0.369477,0.393789,0.0,0.0,0.351877,0.405190,...,0.641477,0.358902,0.647209,0.351591,0.734951,0.334408,0.744310,23.mp4,0,57
2544,0.375842,0.398520,0.372638,0.392600,0.369709,0.393252,0.0,0.0,0.351811,0.404797,...,0.639471,0.360213,0.645472,0.350392,0.734034,0.335498,0.741388,23.mp4,0,58
2545,0.376478,0.398737,0.372547,0.392018,0.370101,0.392954,0.0,0.0,0.351889,0.404138,...,0.638220,0.357322,0.644855,0.353428,0.732414,0.332303,0.740661,23.mp4,0,59
2546,0.376569,0.398836,0.000000,0.000000,0.369938,0.392954,0.0,0.0,0.351718,0.403736,...,0.638895,0.352830,0.644631,0.350808,0.733159,0.334628,0.740569,23.mp4,0,60


In [ ]:
df_all_coord.isna().sum()

0               0
1               0
2               0
3               0
4               0
5               0
6               0
7               0
8               0
9               0
10              0
11              0
12              0
13              0
14              0
15              0
16              0
17              0
18              0
19              0
20              0
21              0
22              0
23              0
24              0
25              0
26              0
27              0
28              0
29              0
30              0
31              0
32              0
33              0
video           0
label           0
frame_number    0
dtype: int64

#### Label the dataset with columns with different parts and their X and Y coordinates

In [ ]:
body_parts = [
'Nose',
'Left_eye',
'Right_eye',
'Left_ear',
'Right_ear',
'Left_shoulder',
'Right_shoulder',
'Left_elbow',
 'Right_elbow',
'Left_wrist',
'Right_wrist',
'Left_hip',
'Right_hip',
'Left_knee',
'Right_knee',
'Left_ankle',
'Right_ankle']

new_body_parts = []

for i in range(0,len(body_parts)):

  new_body_parts.append(body_parts[i] + '_x')
  new_body_parts.append(body_parts[i] + '_y')

num_list=[i for i in range(0,len(new_body_parts))]

print(new_body_parts)
print(num_list)

['Nose_x', 'Nose_y', 'Left_eye_x', 'Left_eye_y', 'Right_eye_x', 'Right_eye_y', 'Left_ear_x', 'Left_ear_y', 'Right_ear_x', 'Right_ear_y', 'Left_shoulder_x', 'Left_shoulder_y', 'Right_shoulder_x', 'Right_shoulder_y', 'Left_elbow_x', 'Left_elbow_y', 'Right_elbow_x', 'Right_elbow_y', 'Left_wrist_x', 'Left_wrist_y', 'Right_wrist_x', 'Right_wrist_y', 'Left_hip_x', 'Left_hip_y', 'Right_hip_x', 'Right_hip_y', 'Left_knee_x', 'Left_knee_y', 'Right_knee_x', 'Right_knee_y', 'Left_ankle_x', 'Left_ankle_y', 'Right_ankle_x', 'Right_ankle_y']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]


#### Make a dictionary to rename columns

In [ ]:
dict_body = {}
for i in range(len(new_body_parts)):
    dict_body[num_list[i]] = new_body_parts[i]
print(dict_body)

{0: 'Nose_x', 1: 'Nose_y', 2: 'Left_eye_x', 3: 'Left_eye_y', 4: 'Right_eye_x', 5: 'Right_eye_y', 6: 'Left_ear_x', 7: 'Left_ear_y', 8: 'Right_ear_x', 9: 'Right_ear_y', 10: 'Left_shoulder_x', 11: 'Left_shoulder_y', 12: 'Right_shoulder_x', 13: 'Right_shoulder_y', 14: 'Left_elbow_x', 15: 'Left_elbow_y', 16: 'Right_elbow_x', 17: 'Right_elbow_y', 18: 'Left_wrist_x', 19: 'Left_wrist_y', 20: 'Right_wrist_x', 21: 'Right_wrist_y', 22: 'Left_hip_x', 23: 'Left_hip_y', 24: 'Right_hip_x', 25: 'Right_hip_y', 26: 'Left_knee_x', 27: 'Left_knee_y', 28: 'Right_knee_x', 29: 'Right_knee_y', 30: 'Left_ankle_x', 31: 'Left_ankle_y', 32: 'Right_ankle_x', 33: 'Right_ankle_y'}


In [ ]:
df_all_coord.rename(columns=dict_body, inplace=True)

df_all_coord

,Nose_x,Nose_y,Left_eye_x,Left_eye_y,Right_eye_x,Right_eye_y,Left_ear_x,Left_ear_y,Right_ear_x,Right_ear_y,...,Left_knee_y,Right_knee_x,Right_knee_y,Left_ankle_x,Left_ankle_y,Right_ankle_x,Right_ankle_y,video,label,frame_number
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,ankur-make-1.mp4,1,1
1,0.141942,0.519379,0.140354,0.512539,0.133615,0.515330,0.0,0.0,0.117914,0.528823,...,0.749542,0.120475,0.763546,0.123502,0.825739,0.101721,0.848228,ankur-make-1.mp4,1,2
2,0.143001,0.522868,0.140956,0.515682,0.134432,0.518720,0.0,0.0,0.117831,0.532366,...,0.750962,0.130295,0.763179,0.125134,0.828403,0.102264,0.846033,ankur-make-1.mp4,1,3
3,0.142824,0.530111,0.000000,0.000000,0.134676,0.526676,0.0,0.0,0.118476,0.540698,...,0.754452,0.135328,0.768395,0.123493,0.830647,0.105687,0.852329,ankur-make-1.mp4,1,4
4,0.145013,0.539606,0.000000,0.000000,0.137419,0.535154,0.0,0.0,0.120887,0.547287,...,0.762864,0.137136,0.777557,0.118015,0.833874,0.108978,0.855727,ankur-make-1.mp4,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2543,0.375570,0.399020,0.372361,0.393344,0.369477,0.393789,0.0,0.0,0.351877,0.405190,...,0.641477,0.358902,0.647209,0.351591,0.734951,0.334408,0.744310,23.mp4,0,57
2544,0.375842,0.398520,0.372638,0.392600,0.369709,0.393252,0.0,0.0,0.351811,0.404797,...,0.639471,0.360213,0.645472,0.350392,0.734034,0.335498,0.741388,23.mp4,0,58
2545,0.376478,0.398737,0.372547,0.392018,0.370101,0.392954,0.0,0.0,0.351889,0.404138,...,0.638220,0.357322,0.644855,0.353428,0.732414,0.332303,0.740661,23.mp4,0,59
2546,0.376569,0.398836,0.000000,0.000000,0.369938,0.392954,0.0,0.0,0.351718,0.403736,...,0.638895,0.352830,0.644631,0.350808,0.733159,0.334628,0.740569,23.mp4,0,60


In [ ]:

df_all_coord_updated=df_all_coord.loc[:,['Left_shoulder_x','Left_shoulder_y','Right_shoulder_x'   , 'Right_shoulder_y','Left_wrist_y','Left_elbow_x','Left_elbow_y','Right_elbow_x','Right_elbow_y','Left_wrist_x','Left_wrist_y','Right_wrist_x','Right_wrist_y','Left_hip_x','Left_hip_y','Right_hip_x','Right_hip_y','Left_knee_x','Left_knee_y', 'Right_knee_x', 'Right_knee_y','Left_ankle_x', 'Left_ankle_y',  'Right_ankle_x', 'Right_ankle_y','frame_number','video','label']]

df_all_coord_updated

,Left_shoulder_x,Left_shoulder_y,Right_shoulder_x,Right_shoulder_y,Left_wrist_y,Left_elbow_x,Left_elbow_y,Right_elbow_x,Right_elbow_y,Left_wrist_x,...,Left_knee_y,Right_knee_x,Right_knee_y,Left_ankle_x,Left_ankle_y,Right_ankle_x,Right_ankle_y,frame_number,video,label
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,ankur-make-1.mp4,1
1,0.137992,0.567018,0.114047,0.574758,0.632627,0.138122,0.620622,0.112386,0.633044,0.163240,...,0.749542,0.120475,0.763546,0.123502,0.825739,0.101721,0.848228,2,ankur-make-1.mp4,1
2,0.137748,0.569186,0.115349,0.579705,0.630857,0.142738,0.619906,0.118926,0.639296,0.169016,...,0.750962,0.130295,0.763179,0.125134,0.828403,0.102264,0.846033,3,ankur-make-1.mp4,1
3,0.135332,0.574709,0.118000,0.584913,0.630535,0.143820,0.624669,0.126229,0.641880,0.169926,...,0.754452,0.135328,0.768395,0.123493,0.830647,0.105687,0.852329,4,ankur-make-1.mp4,1
4,0.137106,0.581849,0.117793,0.593527,0.632422,0.148398,0.628282,0.137886,0.648005,0.175129,...,0.762864,0.137136,0.777557,0.118015,0.833874,0.108978,0.855727,5,ankur-make-1.mp4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2543,0.336772,0.450447,0.361769,0.440221,0.571684,0.315813,0.510262,0.387930,0.452417,0.306920,...,0.641477,0.358902,0.647209,0.351591,0.734951,0.334408,0.744310,57,23.mp4,0
2544,0.336169,0.446795,0.361282,0.442094,0.569260,0.315534,0.508232,0.381513,0.467133,0.306491,...,0.639471,0.360213,0.645472,0.350392,0.734034,0.335498,0.741388,58,23.mp4,0
2545,0.336118,0.446699,0.360174,0.445972,0.566294,0.317613,0.512701,0.379284,0.479733,0.307859,...,0.638220,0.357322,0.644855,0.353428,0.732414,0.332303,0.740661,59,23.mp4,0
2546,0.337864,0.445126,0.358200,0.446341,0.569123,0.318504,0.510571,0.374125,0.480640,0.309391,...,0.638895,0.352830,0.644631,0.350808,0.733159,0.334628,0.740569,60,23.mp4,0


#### Calculate the angles for different body parts. We use (Y2-Y1)/(X2-X1) to get the slope between the 2 coordinates

In [ ]:
def angle_calculator(df,var_from, var_to,angle_var):
  x_var_from=var_from+'_x'
  y_var_from=var_from+'_y'
  x_var_to=var_to+'_x'
  y_var_to=var_to+'_y'
  print(x_var_from,y_var_from,x_var_to,y_var_to)
  df[y_var_to] = df[y_var_to].astype(float)
  df[y_var_from] = df[y_var_from].astype(float)
  df[x_var_to] = df[x_var_to].astype(float)
  df[x_var_from] = df[x_var_from].astype(float)
  angle_rad = np.arctan2(df[y_var_to] - df[y_var_from], df[x_var_to] - df[x_var_from])
  angle_deg = np.degrees(angle_rad)
  df[angle_var]=angle_deg
  return df

In [ ]:
angle_calculator(df_all_coord_updated,'Right_shoulder','Right_elbow',  'Right_shoulder_angle')
df_all_coord_updated

Right_shoulder_x Right_shoulder_y Right_elbow_x Right_elbow_y


,Left_shoulder_x,Left_shoulder_y,Right_shoulder_x,Right_shoulder_y,Left_wrist_y,Left_elbow_x,Left_elbow_y,Right_elbow_x,Right_elbow_y,Left_wrist_x,...,Right_knee_x,Right_knee_y,Left_ankle_x,Left_ankle_y,Right_ankle_x,Right_ankle_y,frame_number,video,label,Right_shoulder_angle
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,ankur-make-1.mp4,1,0.000000
1,0.137992,0.567018,0.114047,0.574758,0.632627,0.138122,0.620622,0.112386,0.633044,0.163240,...,0.120475,0.763546,0.123502,0.825739,0.101721,0.848228,2,ankur-make-1.mp4,1,91.632777
2,0.137748,0.569186,0.115349,0.579705,0.630857,0.142738,0.619906,0.118926,0.639296,0.169016,...,0.130295,0.763179,0.125134,0.828403,0.102264,0.846033,3,ankur-make-1.mp4,1,86.564595
3,0.135332,0.574709,0.118000,0.584913,0.630535,0.143820,0.624669,0.126229,0.641880,0.169926,...,0.135328,0.768395,0.123493,0.830647,0.105687,0.852329,4,ankur-make-1.mp4,1,81.780484
4,0.137106,0.581849,0.117793,0.593527,0.632422,0.148398,0.628282,0.137886,0.648005,0.175129,...,0.137136,0.777557,0.118015,0.833874,0.108978,0.855727,5,ankur-make-1.mp4,1,69.754535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2543,0.336772,0.450447,0.361769,0.440221,0.571684,0.315813,0.510262,0.387930,0.452417,0.306920,...,0.358902,0.647209,0.351591,0.734951,0.334408,0.744310,57,23.mp4,0,24.993885
2544,0.336169,0.446795,0.361282,0.442094,0.569260,0.315534,0.508232,0.381513,0.467133,0.306491,...,0.360213,0.645472,0.350392,0.734034,0.335498,0.741388,58,23.mp4,0,51.062503
2545,0.336118,0.446699,0.360174,0.445972,0.566294,0.317613,0.512701,0.379284,0.479733,0.307859,...,0.357322,0.644855,0.353428,0.732414,0.332303,0.740661,59,23.mp4,0,60.489315
2546,0.337864,0.445126,0.358200,0.446341,0.569123,0.318504,0.510571,0.374125,0.480640,0.309391,...,0.352830,0.644631,0.350808,0.733159,0.334628,0.740569,60,23.mp4,0,65.094371


In [ ]:
angle_calculator(df_all_coord_updated, 'Left_shoulder','Left_elbow','Left_Shoulder_angle')
df_all_coord_updated

Left_shoulder_x Left_shoulder_y Left_elbow_x Left_elbow_y


,Left_shoulder_x,Left_shoulder_y,Right_shoulder_x,Right_shoulder_y,Left_wrist_y,Left_elbow_x,Left_elbow_y,Right_elbow_x,Right_elbow_y,Left_wrist_x,...,Right_knee_y,Left_ankle_x,Left_ankle_y,Right_ankle_x,Right_ankle_y,frame_number,video,label,Right_shoulder_angle,Left_Shoulder_angle
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1,ankur-make-1.mp4,1,0.000000,0.000000
1,0.137992,0.567018,0.114047,0.574758,0.632627,0.138122,0.620622,0.112386,0.633044,0.163240,...,0.763546,0.123502,0.825739,0.101721,0.848228,2,ankur-make-1.mp4,1,91.632777,89.861400
2,0.137748,0.569186,0.115349,0.579705,0.630857,0.142738,0.619906,0.118926,0.639296,0.169016,...,0.763179,0.125134,0.828403,0.102264,0.846033,3,ankur-make-1.mp4,1,86.564595,84.381073
3,0.135332,0.574709,0.118000,0.584913,0.630535,0.143820,0.624669,0.126229,0.641880,0.169926,...,0.768395,0.123493,0.830647,0.105687,0.852329,4,ankur-make-1.mp4,1,81.780484,80.358441
4,0.137106,0.581849,0.117793,0.593527,0.632422,0.148398,0.628282,0.137886,0.648005,0.175129,...,0.777557,0.118015,0.833874,0.108978,0.855727,5,ankur-make-1.mp4,1,69.754535,76.331635
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2543,0.336772,0.450447,0.361769,0.440221,0.571684,0.315813,0.510262,0.387930,0.452417,0.306920,...,0.647209,0.351591,0.734951,0.334408,0.744310,57,23.mp4,0,24.993885,109.311059
2544,0.336169,0.446795,0.361282,0.442094,0.569260,0.315534,0.508232,0.381513,0.467133,0.306491,...,0.645472,0.350392,0.734034,0.335498,0.741388,58,23.mp4,0,51.062503,108.565706
2545,0.336118,0.446699,0.360174,0.445972,0.566294,0.317613,0.512701,0.379284,0.479733,0.307859,...,0.644855,0.353428,0.732414,0.332303,0.740661,59,23.mp4,0,60.489315,105.661801
2546,0.337864,0.445126,0.358200,0.446341,0.569123,0.318504,0.510571,0.374125,0.480640,0.309391,...,0.644631,0.350808,0.733159,0.334628,0.740569,60,23.mp4,0,65.094371,106.478589


In [ ]:
angle_calculator(df_all_coord_updated, 'Right_elbow','Right_wrist','Right_wrist_angle')
df_all_coord_updated

Right_elbow_x Right_elbow_y Right_wrist_x Right_wrist_y


,Left_shoulder_x,Left_shoulder_y,Right_shoulder_x,Right_shoulder_y,Left_wrist_y,Left_elbow_x,Left_elbow_y,Right_elbow_x,Right_elbow_y,Left_wrist_x,...,Left_ankle_x,Left_ankle_y,Right_ankle_x,Right_ankle_y,frame_number,video,label,Right_shoulder_angle,Left_Shoulder_angle,Right_wrist_angle
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1,ankur-make-1.mp4,1,0.000000,0.000000,0.000000
1,0.137992,0.567018,0.114047,0.574758,0.632627,0.138122,0.620622,0.112386,0.633044,0.163240,...,0.123502,0.825739,0.101721,0.848228,2,ankur-make-1.mp4,1,91.632777,89.861400,17.659975
2,0.137748,0.569186,0.115349,0.579705,0.630857,0.142738,0.619906,0.118926,0.639296,0.169016,...,0.125134,0.828403,0.102264,0.846033,3,ankur-make-1.mp4,1,86.564595,84.381073,7.643551
3,0.135332,0.574709,0.118000,0.584913,0.630535,0.143820,0.624669,0.126229,0.641880,0.169926,...,0.123493,0.830647,0.105687,0.852329,4,ankur-make-1.mp4,1,81.780484,80.358441,0.619993
4,0.137106,0.581849,0.117793,0.593527,0.632422,0.148398,0.628282,0.137886,0.648005,0.175129,...,0.118015,0.833874,0.108978,0.855727,5,ankur-make-1.mp4,1,69.754535,76.331635,-15.724972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2543,0.336772,0.450447,0.361769,0.440221,0.571684,0.315813,0.510262,0.387930,0.452417,0.306920,...,0.351591,0.734951,0.334408,0.744310,57,23.mp4,0,24.993885,109.311059,-20.092856
2544,0.336169,0.446795,0.361282,0.442094,0.569260,0.315534,0.508232,0.381513,0.467133,0.306491,...,0.350392,0.734034,0.335498,0.741388,58,23.mp4,0,51.062503,108.565706,-28.560092
2545,0.336118,0.446699,0.360174,0.445972,0.566294,0.317613,0.512701,0.379284,0.479733,0.307859,...,0.353428,0.732414,0.332303,0.740661,59,23.mp4,0,60.489315,105.661801,-40.917997
2546,0.337864,0.445126,0.358200,0.446341,0.569123,0.318504,0.510571,0.374125,0.480640,0.309391,...,0.350808,0.733159,0.334628,0.740569,60,23.mp4,0,65.094371,106.478589,-31.887617


In [ ]:
angle_calculator(df_all_coord_updated, 'Right_hip', 'Right_knee','Right_thigh_angle')
df_all_coord_updated

Right_hip_x Right_hip_y Right_knee_x Right_knee_y


,Left_shoulder_x,Left_shoulder_y,Right_shoulder_x,Right_shoulder_y,Left_wrist_y,Left_elbow_x,Left_elbow_y,Right_elbow_x,Right_elbow_y,Left_wrist_x,...,Left_ankle_y,Right_ankle_x,Right_ankle_y,frame_number,video,label,Right_shoulder_angle,Left_Shoulder_angle,Right_wrist_angle,Right_thigh_angle
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1,ankur-make-1.mp4,1,0.000000,0.000000,0.000000,0.000000
1,0.137992,0.567018,0.114047,0.574758,0.632627,0.138122,0.620622,0.112386,0.633044,0.163240,...,0.825739,0.101721,0.848228,2,ankur-make-1.mp4,1,91.632777,89.861400,17.659975,85.636815
2,0.137748,0.569186,0.115349,0.579705,0.630857,0.142738,0.619906,0.118926,0.639296,0.169016,...,0.828403,0.102264,0.846033,3,ankur-make-1.mp4,1,86.564595,84.381073,7.643551,80.122440
3,0.135332,0.574709,0.118000,0.584913,0.630535,0.143820,0.624669,0.126229,0.641880,0.169926,...,0.830647,0.105687,0.852329,4,ankur-make-1.mp4,1,81.780484,80.358441,0.619993,76.545692
4,0.137106,0.581849,0.117793,0.593527,0.632422,0.148398,0.628282,0.137886,0.648005,0.175129,...,0.833874,0.108978,0.855727,5,ankur-make-1.mp4,1,69.754535,76.331635,-15.724972,78.640070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2543,0.336772,0.450447,0.361769,0.440221,0.571684,0.315813,0.510262,0.387930,0.452417,0.306920,...,0.734951,0.334408,0.744310,57,23.mp4,0,24.993885,109.311059,-20.092856,90.292186
2544,0.336169,0.446795,0.361282,0.442094,0.569260,0.315534,0.508232,0.381513,0.467133,0.306491,...,0.734034,0.335498,0.741388,58,23.mp4,0,51.062503,108.565706,-28.560092,89.391243
2545,0.336118,0.446699,0.360174,0.445972,0.566294,0.317613,0.512701,0.379284,0.479733,0.307859,...,0.732414,0.332303,0.740661,59,23.mp4,0,60.489315,105.661801,-40.917997,91.127912
2546,0.337864,0.445126,0.358200,0.446341,0.569123,0.318504,0.510571,0.374125,0.480640,0.309391,...,0.733159,0.334628,0.740569,60,23.mp4,0,65.094371,106.478589,-31.887617,92.397735


In [ ]:
angle_calculator( df_all_coord_updated,'Left_hip', 'Left_knee','Left_thigh_angle')
df_all_coord_updated

Left_hip_x Left_hip_y Left_knee_x Left_knee_y


,Left_shoulder_x,Left_shoulder_y,Right_shoulder_x,Right_shoulder_y,Left_wrist_y,Left_elbow_x,Left_elbow_y,Right_elbow_x,Right_elbow_y,Left_wrist_x,...,Right_ankle_x,Right_ankle_y,frame_number,video,label,Right_shoulder_angle,Left_Shoulder_angle,Right_wrist_angle,Right_thigh_angle,Left_thigh_angle
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1,ankur-make-1.mp4,1,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.137992,0.567018,0.114047,0.574758,0.632627,0.138122,0.620622,0.112386,0.633044,0.163240,...,0.101721,0.848228,2,ankur-make-1.mp4,1,91.632777,89.861400,17.659975,85.636815,81.147934
2,0.137748,0.569186,0.115349,0.579705,0.630857,0.142738,0.619906,0.118926,0.639296,0.169016,...,0.102264,0.846033,3,ankur-make-1.mp4,1,86.564595,84.381073,7.643551,80.122440,78.685173
3,0.135332,0.574709,0.118000,0.584913,0.630535,0.143820,0.624669,0.126229,0.641880,0.169926,...,0.105687,0.852329,4,ankur-make-1.mp4,1,81.780484,80.358441,0.619993,76.545692,71.493804
4,0.137106,0.581849,0.117793,0.593527,0.632422,0.148398,0.628282,0.137886,0.648005,0.175129,...,0.108978,0.855727,5,ankur-make-1.mp4,1,69.754535,76.331635,-15.724972,78.640070,75.378355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2543,0.336772,0.450447,0.361769,0.440221,0.571684,0.315813,0.510262,0.387930,0.452417,0.306920,...,0.334408,0.744310,57,23.mp4,0,24.993885,109.311059,-20.092856,90.292186,83.867412
2544,0.336169,0.446795,0.361282,0.442094,0.569260,0.315534,0.508232,0.381513,0.467133,0.306491,...,0.335498,0.741388,58,23.mp4,0,51.062503,108.565706,-28.560092,89.391243,83.985483
2545,0.336118,0.446699,0.360174,0.445972,0.566294,0.317613,0.512701,0.379284,0.479733,0.307859,...,0.332303,0.740661,59,23.mp4,0,60.489315,105.661801,-40.917997,91.127912,82.837666
2546,0.337864,0.445126,0.358200,0.446341,0.569123,0.318504,0.510571,0.374125,0.480640,0.309391,...,0.334628,0.740569,60,23.mp4,0,65.094371,106.478589,-31.887617,92.397735,87.047832


In [ ]:
angle_calculator(df_all_coord_updated, 'Left_knee', 'Left_ankle','Left_femur_angle')
df_all_coord_updated

Left_knee_x Left_knee_y Left_ankle_x Left_ankle_y


,Left_shoulder_x,Left_shoulder_y,Right_shoulder_x,Right_shoulder_y,Left_wrist_y,Left_elbow_x,Left_elbow_y,Right_elbow_x,Right_elbow_y,Left_wrist_x,...,Right_ankle_y,frame_number,video,label,Right_shoulder_angle,Left_Shoulder_angle,Right_wrist_angle,Right_thigh_angle,Left_thigh_angle,Left_femur_angle
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1,ankur-make-1.mp4,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.137992,0.567018,0.114047,0.574758,0.632627,0.138122,0.620622,0.112386,0.633044,0.163240,...,0.848228,2,ankur-make-1.mp4,1,91.632777,89.861400,17.659975,85.636815,81.147934,104.047748
2,0.137748,0.569186,0.115349,0.579705,0.630857,0.142738,0.619906,0.118926,0.639296,0.169016,...,0.846033,3,ankur-make-1.mp4,1,86.564595,84.381073,7.643551,80.122440,78.685173,105.756077
3,0.135332,0.574709,0.118000,0.584913,0.630535,0.143820,0.624669,0.126229,0.641880,0.169926,...,0.852329,4,ankur-make-1.mp4,1,81.780484,80.358441,0.619993,76.545692,71.493804,111.111186
4,0.137106,0.581849,0.117793,0.593527,0.632422,0.148398,0.628282,0.137886,0.648005,0.175129,...,0.855727,5,ankur-make-1.mp4,1,69.754535,76.331635,-15.724972,78.640070,75.378355,117.127077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2543,0.336772,0.450447,0.361769,0.440221,0.571684,0.315813,0.510262,0.387930,0.452417,0.306920,...,0.744310,57,23.mp4,0,24.993885,109.311059,-20.092856,90.292186,83.867412,93.736411
2544,0.336169,0.446795,0.361282,0.442094,0.569260,0.315534,0.508232,0.381513,0.467133,0.306491,...,0.741388,58,23.mp4,0,51.062503,108.565706,-28.560092,89.391243,83.985483,93.651813
2545,0.336118,0.446699,0.360174,0.445972,0.566294,0.317613,0.512701,0.379284,0.479733,0.307859,...,0.740661,59,23.mp4,0,60.489315,105.661801,-40.917997,91.127912,82.837666,93.438897
2546,0.337864,0.445126,0.358200,0.446341,0.569123,0.318504,0.510571,0.374125,0.480640,0.309391,...,0.740569,60,23.mp4,0,65.094371,106.478589,-31.887617,92.397735,87.047832,90.745309


In [ ]:
angle_calculator(df_all_coord_updated, 'Right_knee', 'Right_ankle','Right_femur_angle')
df_all_coord_updated

Right_knee_x Right_knee_y Right_ankle_x Right_ankle_y


,Left_shoulder_x,Left_shoulder_y,Right_shoulder_x,Right_shoulder_y,Left_wrist_y,Left_elbow_x,Left_elbow_y,Right_elbow_x,Right_elbow_y,Left_wrist_x,...,frame_number,video,label,Right_shoulder_angle,Left_Shoulder_angle,Right_wrist_angle,Right_thigh_angle,Left_thigh_angle,Left_femur_angle,Right_femur_angle
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1,ankur-make-1.mp4,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.137992,0.567018,0.114047,0.574758,0.632627,0.138122,0.620622,0.112386,0.633044,0.163240,...,2,ankur-make-1.mp4,1,91.632777,89.861400,17.659975,85.636815,81.147934,104.047748,102.487828
2,0.137748,0.569186,0.115349,0.579705,0.630857,0.142738,0.619906,0.118926,0.639296,0.169016,...,3,ankur-make-1.mp4,1,86.564595,84.381073,7.643551,80.122440,78.685173,105.756077,108.691545
3,0.135332,0.574709,0.118000,0.584913,0.630535,0.143820,0.624669,0.126229,0.641880,0.169926,...,4,ankur-make-1.mp4,1,81.780484,80.358441,0.619993,76.545692,71.493804,111.111186,109.450115
4,0.137106,0.581849,0.117793,0.593527,0.632422,0.148398,0.628282,0.137886,0.648005,0.175129,...,5,ankur-make-1.mp4,1,69.754535,76.331635,-15.724972,78.640070,75.378355,117.127077,109.809570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2543,0.336772,0.450447,0.361769,0.440221,0.571684,0.315813,0.510262,0.387930,0.452417,0.306920,...,57,23.mp4,0,24.993885,109.311059,-20.092856,90.292186,83.867412,93.736411,104.157638
2544,0.336169,0.446795,0.361282,0.442094,0.569260,0.315534,0.508232,0.381513,0.467133,0.306491,...,58,23.mp4,0,51.062503,108.565706,-28.560092,89.391243,83.985483,93.651813,104.449236
2545,0.336118,0.446699,0.360174,0.445972,0.566294,0.317613,0.512701,0.379284,0.479733,0.307859,...,59,23.mp4,0,60.489315,105.661801,-40.917997,91.127912,82.837666,93.438897,104.635814
2546,0.337864,0.445126,0.358200,0.446341,0.569123,0.318504,0.510571,0.374125,0.480640,0.309391,...,60,23.mp4,0,65.094371,106.478589,-31.887617,92.397735,87.047832,90.745309,100.742641


In [ ]:
angle_calculator( df_all_coord_updated,'Left_hip', 'Right_hip','hip_angle')
df_all_coord_updated

Left_hip_x Left_hip_y Right_hip_x Right_hip_y


,Left_shoulder_x,Left_shoulder_y,Right_shoulder_x,Right_shoulder_y,Left_wrist_y,Left_elbow_x,Left_elbow_y,Right_elbow_x,Right_elbow_y,Left_wrist_x,...,video,label,Right_shoulder_angle,Left_Shoulder_angle,Right_wrist_angle,Right_thigh_angle,Left_thigh_angle,Left_femur_angle,Right_femur_angle,hip_angle
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,ankur-make-1.mp4,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.137992,0.567018,0.114047,0.574758,0.632627,0.138122,0.620622,0.112386,0.633044,0.163240,...,ankur-make-1.mp4,1,91.632777,89.861400,17.659975,85.636815,81.147934,104.047748,102.487828,159.689104
2,0.137748,0.569186,0.115349,0.579705,0.630857,0.142738,0.619906,0.118926,0.639296,0.169016,...,ankur-make-1.mp4,1,86.564595,84.381073,7.643551,80.122440,78.685173,105.756077,108.691545,156.412974
3,0.135332,0.574709,0.118000,0.584913,0.630535,0.143820,0.624669,0.126229,0.641880,0.169926,...,ankur-make-1.mp4,1,81.780484,80.358441,0.619993,76.545692,71.493804,111.111186,109.450115,149.968763
4,0.137106,0.581849,0.117793,0.593527,0.632422,0.148398,0.628282,0.137886,0.648005,0.175129,...,ankur-make-1.mp4,1,69.754535,76.331635,-15.724972,78.640070,75.378355,117.127077,109.809570,150.714713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2543,0.336772,0.450447,0.361769,0.440221,0.571684,0.315813,0.510262,0.387930,0.452417,0.306920,...,23.mp4,0,24.993885,109.311059,-20.092856,90.292186,83.867412,93.736411,104.157638,-8.234357
2544,0.336169,0.446795,0.361282,0.442094,0.569260,0.315534,0.508232,0.381513,0.467133,0.306491,...,23.mp4,0,51.062503,108.565706,-28.560092,89.391243,83.985483,93.651813,104.449236,0.830669
2545,0.336118,0.446699,0.360174,0.445972,0.566294,0.317613,0.512701,0.379284,0.479733,0.307859,...,23.mp4,0,60.489315,105.661801,-40.917997,91.127912,82.837666,93.438897,104.635814,14.399112
2546,0.337864,0.445126,0.358200,0.446341,0.569123,0.318504,0.510571,0.374125,0.480640,0.309391,...,23.mp4,0,65.094371,106.478589,-31.887617,92.397735,87.047832,90.745309,100.742641,21.186660
